In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
prompt_template = ChatPromptTemplate([
    ('system','''
    이곳이 처음이거나 자주 오는 이들에게도 모르는 것이 있다면 언제든 대답해주는 게 네 역할이지.



마사회
You are a Virtual Assistant AI designed to serve as an information desk guide for the Korea Racing Authority (KRA). Your primary role is to provide clear, friendly, and detailed answers to visitors’ questions. Follow these steps when assisting users:

Step 1: Document Analysis

Review the uploaded documents or reference materials provided to identify relevant information regarding the user’s question.

Step 2: Database Search or Supplementary Research

If the documents lack sufficient details, search through available internal databases or reference reliable sources to gather the required information.

Step 3: Answer Formulation

Combine the extracted data to craft a concise, accurate, and visitor-friendly response. Tailor your tone to be professional yet approachable.

Step 4: Simplification and Clarity

Ensure the information is easy to understand, avoiding technical jargon unless specifically requested by the user.

Step 5: Summary and Organization

Provide a brief summary of your answer and, where applicable, suggest any additional services or information that might enhance the visitor’s experience.

Now, proceed to execute this task: “Provide answers to visitor inquiries based on available documents and additional resources.” Take a step-by-step approach to ensure the response is comprehensive and accurate.''' ),
    
    ('user', '{question}')
])

# 출력 파서 생성
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

## pdf 파일 로드

In [38]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain_community.document_loaders import PyPDFLoader
import os

# 여러 PDF 파일 경로를 리스트로 생성
pdf_folder = 'C:/Workspaces/project_ma/pdfs'
pdf_files = [os.path.join(pdf_folder, file) for file in os.listdir(pdf_folder) if file.endswith('.pdf')]

# 각 PDF 파일을 로드하여 문서로 처리
pdf_documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents.extend(loader.load())  # 문서를 리스트에 추가

print(f"Loaded {len(pdf_documents)} documents.")

Loaded 203 documents.


In [3]:
from langchain_community.document_loaders import PyPDFLoader

# 각 PDF 파일을 로드하여 문서로 처리
pdf_documents = []
for pdf_file in ['C:\\Workspaces\\project_ma\\pdfs\\race_guide.pdf']:
    loader = PyPDFLoader(pdf_file)
    pdf_documents.extend(loader.load())
    
pdf_documents

[Document(metadata={'source': 'C:\\Workspaces\\project_ma\\pdfs\\race_guide.pdf', 'page': 0}, page_content='경주결과를 보는 방법(순위게시대)현재 운영 중인 베팅방법 7가지경마진행 이해하기\n부산경남경마공원 대표마\n배당률 화면 보는 방법\n오토벳(Auto Bet)\n경주가 끝난 후 결과를 알려주는 순위게시대\n1등에서 5등까지 순위를 발표합니다.\n착차(순위 간 차이)란 결승선을 통과할 때 선행말과\n후속말 간의 거리차를 말합니다.\n우승마의 예상이 어려우신 경우 Auto Bet 즉, \n자동선택방식을 이용해 보세요.\n● 선택원리: 구매시점 기준 단승식 저배당 5위 내 임의선택\n● 구매방법: 구매표 상 ‘마번’ 대신 ‘자동(A)’칸에 가입·신청\n연승식·복연승식\n연승식: 구매마권이 연승식 8번인 경우 ▶7.0배의 배당률\n복연승식: 구매마권이 복연승식 4번, 8번일 경우 ▶9.4배의 배당률\n코차 (약 0.1~21cm)\n머리차 (약 22~40cm)\n목차 (약  52~100cm)\n1/2마신차 (약 1.2m)\n3/4마신차 (약 1.8m)\n1마신차 (약 2.4m)\n대차 (10마신 이상)\n고속카메라로  1,500\n1\n 초까지 식별합니다.\n1마신은 0.18초, 6마신은 1초로 환산됩니다.\n초보자를 위한\n경마가이드\n착차 표시 기준\n경주번호 경주거리\n심의 중 ‘녹색’ 점등\n착차:   결승선 먼저 도달한 말의 코끝과\n 다음에 도달한 말의 코끝 차이 \n 코, 머리, 목, 그 이상은 마신 단위로 표시\n4펄롱\n(결승 전방 800m 부터 결승선까지 1등마 기록)\n3펄롱\n(결승 전방 600m 부터 결승선까지 1등마 기록)\n확정 시 ‘적색’ 점등\n순위 ‘1~5등’ 표시\n비워두세요!\n① 배당방식 선택\n②자동(A) 선택\n③구입하고 싶은 금액\n부산경남 대표 가족공원, 말테마파크www.kra.co.kr\n 구매 상한선 준수, \n 건강한 여가생활의 시

In [4]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [5]:
from langchain.vectorstores import FAISS

vectors = FAISS.from_documents(pdf_documents, embeddings) #documents : Document
vectors

In [11]:
results = vectors.similarity_search("오토벳 ", k=1)
print(len(results))
for i, doc in enumerate(results):
    print(i, doc)


1
0 page_content='경주결과를 보는 방법(순위게시대)현재 운영 중인 베팅방법 7가지경마진행 이해하기
부산경남경마공원 대표마
배당률 화면 보는 방법
오토벳(Auto Bet)
경주가 끝난 후 결과를 알려주는 순위게시대
1등에서 5등까지 순위를 발표합니다.
착차(순위 간 차이)란 결승선을 통과할 때 선행말과
후속말 간의 거리차를 말합니다.
우승마의 예상이 어려우신 경우 Auto Bet 즉, 
자동선택방식을 이용해 보세요.
● 선택원리: 구매시점 기준 단승식 저배당 5위 내 임의선택
● 구매방법: 구매표 상 ‘마번’ 대신 ‘자동(A)’칸에 가입·신청
연승식·복연승식
연승식: 구매마권이 연승식 8번인 경우 ▶7.0배의 배당률
복연승식: 구매마권이 복연승식 4번, 8번일 경우 ▶9.4배의 배당률
코차 (약 0.1~21cm)
머리차 (약 22~40cm)
목차 (약  52~100cm)
1/2마신차 (약 1.2m)
3/4마신차 (약 1.8m)
1마신차 (약 2.4m)
대차 (10마신 이상)
고속카메라로  1,500
1
 초까지 식별합니다.
1마신은 0.18초, 6마신은 1초로 환산됩니다.
초보자를 위한
경마가이드
착차 표시 기준
경주번호 경주거리
심의 중 ‘녹색’ 점등
착차:   결승선 먼저 도달한 말의 코끝과
 다음에 도달한 말의 코끝 차이 
 코, 머리, 목, 그 이상은 마신 단위로 표시
4펄롱
(결승 전방 800m 부터 결승선까지 1등마 기록)
3펄롱
(결승 전방 600m 부터 결승선까지 1등마 기록)
확정 시 ‘적색’ 점등
순위 ‘1~5등’ 표시
비워두세요!
① 배당방식 선택
②자동(A) 선택
③구입하고 싶은 금액
부산경남 대표 가족공원, 말테마파크www.kra.co.kr
 구매 상한선 준수, 
 건강한 여가생활의 시작입니다.46745 부산광역시 강서구 가락대로 929
단  승 식 1등 말을 적중시키는 베팅방식
연  승 식  1, 2, 3등 내에 들어올 말 중 1두 적중시키는 베팅 
방식(출주마가 7두 이하인 경우 1, 2 등 내에 들어온 말만 적중)
복 

## CSV 파일 로드

import os
from langchain_community.document_loaders import CSVLoader

# CSV 저장된 폴더 경로 지정
csv_folder = 'C:/Workspaces/dl_nlp/Choisoovin/10_langchain/04_ai_Sommelier_rag/csvs'

# 폴더 내 모든 CSV 파일 경로 리스트 생성
csv_files = [os.path.join(csv_folder, file) for file in os.listdir(csv_folder) if file.endswith('.csv')]

# 폴더 내 모든 CSV 파일을 Document 객체로 로드
csv_documents = []
for csv_file in csv_files:
    loader = CSVLoader(file_path=csv_file)  
    csv_documents.extend(loader.load())    


## web 방식

from langchain.document_loaders import WebBaseLoader

# 조회하고 싶은 URL들의 리스트
urls = [
    "http://www.okrace.co.kr/okrace_guide/race_guide1.php",
    "http://www.okrace.co.kr/okrace_guide/race_guide2.php",
    "http://www.okrace.co.kr/okrace_guide/race_guide9.php"
]

# WebBaseLoader에 리스트 전달
loader = WebBaseLoader(urls)

# 로드된 문서 가져오기
web_documents = loader.load()

# 출력 확인
len(web_documents)

web_documents[1]

## 데이터 병합

In [13]:
from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
docs = [document.page_content for document in pdf_documents]

all_documents = pdf_documents #+ web_documents

In [14]:
all_documents[19:21]

[]

In [33]:
for doc in all_documents[19:21]:
    print(doc.page_content)

경주결과를 보는 방법(순위게시대)현재 운영 중인 베팅방법 7가지경마진행 이해하기
부산경남경마공원 대표마
배당률 화면 보는 방법
오토벳(Auto Bet)
경주가 끝난 후 결과를 알려주는 순위게시대
1등에서 5등까지 순위를 발표합니다.
착차(순위 간 차이)란 결승선을 통과할 때 선행말과
후속말 간의 거리차를 말합니다.
우승마의 예상이 어려우신 경우 Auto Bet 즉, 
자동선택방식을 이용해 보세요.
● 선택원리: 구매시점 기준 단승식 저배당 5위 내 임의선택
● 구매방법: 구매표 상 ‘마번’ 대신 ‘자동(A)’칸에 가입·신청
연승식·복연승식
연승식: 구매마권이 연승식 8번인 경우 ▶7.0배의 배당률
복연승식: 구매마권이 복연승식 4번, 8번일 경우 ▶9.4배의 배당률
코차 (약 0.1~21cm)
머리차 (약 22~40cm)
목차 (약  52~100cm)
1/2마신차 (약 1.2m)
3/4마신차 (약 1.8m)
1마신차 (약 2.4m)
대차 (10마신 이상)
고속카메라로  1,500
1
 초까지 식별합니다.
1마신은 0.18초, 6마신은 1초로 환산됩니다.
초보자를 위한
경마가이드
착차 표시 기준
경주번호 경주거리
심의 중 ‘녹색’ 점등
착차:   결승선 먼저 도달한 말의 코끝과
 다음에 도달한 말의 코끝 차이 
 코, 머리, 목, 그 이상은 마신 단위로 표시
4펄롱
(결승 전방 800m 부터 결승선까지 1등마 기록)
3펄롱
(결승 전방 600m 부터 결승선까지 1등마 기록)
확정 시 ‘적색’ 점등
순위 ‘1~5등’ 표시
비워두세요!
① 배당방식 선택
②자동(A) 선택
③구입하고 싶은 금액
부산경남 대표 가족공원, 말테마파크www.kra.co.kr
 구매 상한선 준수, 
 건강한 여가생활의 시작입니다.46745 부산광역시 강서구 가락대로 929
단  승 식 1등 말을 적중시키는 베팅방식
연  승 식  1, 2, 3등 내에 들어올 말 중 1두 적중시키는 베팅 
방식(출주마가 7두 이하인 경우 1, 2 등 내에 들어온 말만 적중)
복  승 식  1등 말과 2등 말을 

## Vector Store 구축 - FAISS 사용

In [12]:
from langchain.vectorstores import FAISS

vectors = FAISS.from_documents(all_documents, embeddings) #documents : Document
vectors

NameError: name 'all_documents' is not defined

In [51]:
results = vectors.similarity_search("부산", k=3)
print(len(results))
# for i, doc in enumerate(results):
#     print(i, doc.page_content)

print(results[2])

3
page_content='목   차Ⅰ. 기본 방향· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 1Ⅱ. 2024년 주요 제도 개선사항· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 2Ⅲ. 2024년 경마 시행개요· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 9Ⅳ. 경마 시행체계· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 11  1. 경주 편성· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·11  2. 경주거리 운영· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 12  3. 부담중량 종류 및 운영· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·

## Retriever 객체 생성

In [36]:
retriever = vectors.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F770EF0C80>, search_kwargs={})

In [37]:
# 벡터스토어 검색 및 LLM 질의

from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0.3,
)

#질의 객체
rtvr_qa=RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type = 'stuff' #검색된 내용을 하나의 텍스트로 생성해서 LLM에 전달
)

#llm 질의
response1 = rtvr_qa.invoke('오토벳')
print(response1)

{'query': '오토벳', 'result': '죄송하지만, "오토벳"에 대한 구체적인 정보를 제공할 수 없습니다. 추가적인 정보를 제공해 주시면 도움이 될 수 있습니다.'}


In [ ]:
#response1 = rtvr_qa.invoke('오토벳에 대해 알려줘')
#print(response1)
#TODO pdf 업로드 시, 내용이 누락될 가능성?
#race_guide.pdf에 오토벳에 대한 내용이 있는데 뱉어주는 게 없다.
#민하님께서 guide.pdf만 넣고 돌렸을 땐 관련 내용이 잘 나왔는데, 여러 pdf 파일을 넣고 돌리니깐 해당 내용을 제공할 수 없단 답변을 받았다고 하심.
#이에 다 같이 돌려봤으나 똑같은 증상을 겪었다.